
# Analysis Notebook

This notebook can be used to calculate statistical metrics for the data produced. 
*** 

## 0. Setup <a class="anchor" id="0"></a>
This section imports all files and sets up the notebook.

In [104]:
# Import all pacakges
import os
import pandas as pd
import numpy as np
from scipy import stats


In [105]:
# Choose which results to use
results_to_use = ["Adaptive", "Random"]  # If this is empty, all files in the folder will be used
WARMUP_EPOCHS = 0 # Number of epochs to ignore.
sim_colours = ['blue', 'green', 'red', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']


all_results = os.listdir('results/')

if results_to_use == [""]:
    results_to_use = all_results
# remove DS_Store from the list
if '.DS_Store' in results_to_use:
    results_to_use.remove('.DS_Store')
    
print("Using results: " + str(results_to_use))


Using results: ['Adaptive', 'Random']


In [106]:
# Create a dictionary. Each key is a set of results. 

all_sim_results = {}

for sim in results_to_use:
    all_sim_results[sim] = {}
    all_sim_results[sim]['configuration'] = pd.read_csv(
        'results/' + sim + '/configuration.txt', header=None, delimiter=":")
    types_of_results = []
    # Get all the filenames in the folder, excluding the extension
    for file in os.listdir('results/' + sim + '/exported_data/'):
        if file.endswith(".npy"):
            types_of_results.append(file[:-4])
    # Create a dictionary for each type of result
    for result in types_of_results:
        all_sim_results[sim][result] = np.load(
            'results/' + sim + '/exported_data/' + result + '.npy')

In [107]:
# Show all the all_sim_results in the notebook
for set_of_results in results_to_use:
    print("Results: " + set_of_results)
    print(all_sim_results[set_of_results]['configuration'])

Results: Adaptive
                                    0                    1
0                             command                  run
1                       num_of_epochs                10000
2                  num_of_simulations                  100
3                           grid_size                    3
4                      queue_capacity                   10
5                     congestion_rate                 0.07
6                       with_hotspots                 True
7                           wage_time                   10
8                      credit_balance                    5
9           shared_auction_parameters                 None
10  adaptive_auction_action_selection   e_greedy_exp_decay
11               bid_calculation_rule               linear
12       adaptive_auction_update_rule        simple_bandit
13             auction_episode_length                   10
14   action_selection_hyperparameters                 None
15    adaptive_auction_discretization 

## 1. Average Total Number of Trips  <a class="anchor" id="1"></a>

This t-test compares the average total number of trips between the two experiments, for all sims.

In [108]:
all_average_number_of_trips = []
for sim in results_to_use:
    all_average_number_of_trips.append(
        all_sim_results[sim]['stat_num_of_trips_per_simulation'])

results = stats.ttest_ind(all_average_number_of_trips[0], all_average_number_of_trips[1])
print(results)
#present the results nicely
print("The average number of trips for the first two simulations are: " + str(np.mean(all_average_number_of_trips[0])) + " and " + str(np.mean(all_average_number_of_trips[1])))
print("the standard deviation of number of trips for the first two simulations are: " + str(np.std(all_average_number_of_trips[0])) + " and " + str(np.std(all_average_number_of_trips[1])))
mean_1 = round(np.mean(all_average_number_of_trips[0]),3)
mean_2 = round(np.mean(all_average_number_of_trips[1]),3)
std_1 = round(np.std(all_average_number_of_trips[0]),3)
std_2 = round(np.std(all_average_number_of_trips[1]),3)
print("in latex:","$(M=",mean_1,", SD=",std_1,")$","$(M=",mean_2,", SD=",std_2,")$")
print("in latex:", "t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3))



TtestResult(statistic=5.652413265893216, pvalue=5.4610305922159177e-08, df=198.0)
The average number of trips for the first two simulations are: 26645.57 and 26498.03
the standard deviation of number of trips for the first two simulations are: 187.71687484080914 and 179.48021924435014
in latex: $(M= 26645.57 , SD= 187.717 )$ $(M= 26498.03 , SD= 179.48 )$
in latex: t( 198.0 )= 5.652 , p= 0.0


## 2. Average Congestion of Central Intersection

This t-test is between the 2 experiments, for all sims. It does not process individual epochs

In [109]:
average_congestion_per_intersection = []
for exp in results_to_use:
    average_congestion_per_intersection.append(
        all_sim_results[exp]['stat_average_congestion_per_intersection'])

results = stats.ttest_ind(average_congestion_per_intersection[0][:,1,1], average_congestion_per_intersection[1][:,1,1])
print(results)
print("The average congestion for the first two simulations are: " + str(np.mean(average_congestion_per_intersection[0][:,1,1])) + " and " + str(np.mean(average_congestion_per_intersection[1][:,1,1])))
print("The p-value is: " + str(results[1]))
if results[1] < 0.05:
    print("The difference is significant")
else:
    print("The difference is not significant")
    
mean_1 = round(np.mean(average_congestion_per_intersection[0][:,1,1]),3)
mean_2 = round(np.mean(average_congestion_per_intersection[1][:,1,1]),3)
std_1 = round(np.std(average_congestion_per_intersection[0][:,1,1]),3)
std_2 = round(np.std(average_congestion_per_intersection[1][:,1,1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")

TtestResult(statistic=-8.383781226532703, pvalue=9.470361555500823e-15, df=198.0)
The average congestion for the first two simulations are: 0.37817357499999993 and 0.38611989999999985
The p-value is: 9.470361555500823e-15
The difference is significant
in latex: $(M= 0.378 ,\ SD= 0.008 )$ $(M= 0.386 ,\ SD= 0.006 )$
in latex: $t( 198.0 )= -8.384 , p= 0.0 $


## 3. Average Time Waited <a id="3"></a>

agent, intersection and grid based

In [110]:
######Agent based######
# Average time
average_time_agent = []
for exp in results_to_use:
    average_time_agent.append(
        all_sim_results[exp]['stat_average_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(average_time_agent[0], average_time_agent[1])
print(results)
print("The average time waited for the first two simulations are: " + str(np.mean(average_time_agent[0])) + " and " + str(np.mean(average_time_agent[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(average_time_agent[0]),3)
mean_2 = round(np.mean(average_time_agent[1]),3)
std_1 = round(np.std(average_time_agent[0]),3)
std_2 = round(np.std(average_time_agent[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")

# Max time
max_time_agent = []
for exp in results_to_use:
    max_time_agent.append(
        all_sim_results[exp]['stat_max_time_waited_per_simulation_agent'])
    
results = stats.ttest_ind(max_time_agent[0], max_time_agent[1])
print(results)
print("The max time waited for the first two simulations are: " + str(np.mean(max_time_agent[0])) + " and " + str(np.mean(max_time_agent[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(max_time_agent[0]),3)
mean_2 = round(np.mean(max_time_agent[1]),3)
std_1 = round(np.std(max_time_agent[0]),3)
std_2 = round(np.std(max_time_agent[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ",\ p=", round(results[1], 3), "$")

TtestResult(statistic=-66.34205938495288, pvalue=3.3587674002994525e-137, df=198.0)
The average time waited for the first two simulations are: 0.23138099999999998 and 0.27220061111111105
The p-value is: 3.3587674002994525e-137
in latex: $(M= 0.231 ,\ SD= 0.005 )$ $(M= 0.272 ,\ SD= 0.004 )$
in latex: $t( 198.0 )= -66.342 , p= 0.0 $
TtestResult(statistic=-64.46715881788059, pvalue=7.635159006413398e-135, df=198.0)
The max time waited for the first two simulations are: 0.5864358888888889 and 0.6891041111111111
The p-value is: 7.635159006413398e-135
in latex: $(M= 0.586 ,\ SD= 0.012 )$ $(M= 0.689 ,\ SD= 0.01 )$
in latex: $t( 198.0 )= -64.467 ,\ p= 0.0 $


In [111]:
######Intersection based - Average Time######

average_time_intersection = []
for exp in results_to_use:
    average_time_intersection.append(
        all_sim_results[exp]['stat_average_time_waited_per_intersection'])
    
for i in range(len(average_time_intersection[0][0])):
    for j in range(len(average_time_intersection[0][0])):
        # print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(average_time_intersection[0][:,i,j], average_time_intersection[1][:,i,j])
        # print(results)
        # print("The average time waited for the first two simulations are: " + str(np.mean(average_time_intersection[0][:,i,j])) + " and " + str(np.mean(average_time_intersection[1][:,i,j])))
        # print("The p-value is: " + str(results[1]))
        mean_1 = round(np.mean(average_time_intersection[0][:,i,j]),3)
        mean_2 = round(np.mean(average_time_intersection[1][:,i,j]),3)
        std_1 = round(np.std(average_time_intersection[0][:,i,j]),3)
        std_2 = round(np.std(average_time_intersection[1][:,i,j]),3)
        print("\item [{[", i, ",", j, "]}]: ", "Adaptive $(M=",mean_1,",\ SD=",std_1,")$",", Zero $(M=",mean_2,",\ SD=",std_2,"),\ t(", int(results.df), ")=", round(results[0], 2), ",\ p=", round(results[1], 3), "$")
        # \item [{[0, 0]}]: Adaptive $(M= 0.027 ,\ SD= 0.005 )$ , Random $(M= 0.024 ,\ SD= 0.004 ),$\newline$\ t( 198 )= 5.81 ,\ p<.001 $


\item [{[ 0 , 0 ]}]:  Adaptive $(M= 0.024 ,\ SD= 0.002 )$ , Zero $(M= 0.024 ,\ SD= 0.002 ),\ t( 198 )= 0.36 ,\ p= 0.722 $
\item [{[ 0 , 1 ]}]:  Adaptive $(M= 0.185 ,\ SD= 0.02 )$ , Zero $(M= 0.182 ,\ SD= 0.016 ),\ t( 198 )= 1.14 ,\ p= 0.255 $
\item [{[ 0 , 2 ]}]:  Adaptive $(M= 0.025 ,\ SD= 0.002 )$ , Zero $(M= 0.024 ,\ SD= 0.002 ),\ t( 198 )= 2.98 ,\ p= 0.003 $
\item [{[ 1 , 0 ]}]:  Adaptive $(M= 0.18 ,\ SD= 0.02 )$ , Zero $(M= 0.181 ,\ SD= 0.017 ),\ t( 198 )= -0.42 ,\ p= 0.677 $
\item [{[ 1 , 1 ]}]:  Adaptive $(M= 1.198 ,\ SD= 0.057 )$ , Zero $(M= 1.566 ,\ SD= 0.034 ),\ t( 198 )= -55.42 ,\ p= 0.0 $
\item [{[ 1 , 2 ]}]:  Adaptive $(M= 0.223 ,\ SD= 0.027 )$ , Zero $(M= 0.223 ,\ SD= 0.018 ),\ t( 198 )= -0.03 ,\ p= 0.979 $
\item [{[ 2 , 0 ]}]:  Adaptive $(M= 0.025 ,\ SD= 0.002 )$ , Zero $(M= 0.024 ,\ SD= 0.002 ),\ t( 198 )= 1.86 ,\ p= 0.064 $
\item [{[ 2 , 1 ]}]:  Adaptive $(M= 0.2 ,\ SD= 0.018 )$ , Zero $(M= 0.204 ,\ SD= 0.017 ),\ t( 198 )= -1.49 ,\ p= 0.138 $
\item [{[ 2 , 2 ]}]:  Adap

In [112]:
######Intersection based - Max Time######

max_time_intersection = []
for exp in results_to_use:
    max_time_intersection.append(
        all_sim_results[exp]['stat_max_time_waited_per_intersection'])
    
for i in range(len(max_time_intersection[0][0])):
    for j in range(len(max_time_intersection[0][0])):
        results = stats.ttest_ind(max_time_intersection[0][:,i,j], max_time_intersection[1][:,i,j])
        mean_1 = round(np.mean(max_time_intersection[0][:,i,j]),3)
        mean_2 = round(np.mean(max_time_intersection[1][:,i,j]),3)
        std_1 = round(np.std(max_time_intersection[0][:,i,j]),3)
        std_2 = round(np.std(max_time_intersection[1][:,i,j]),3)
        print("\item [{[", i, ",", j, "]}]: ", "Adaptive $(M=",mean_1,",\ SD=",std_1,")$",", Zero $(M=",mean_2,",\ SD=",std_2,"),\ t(", int(results.df), ")=", round(results[0], 2), ",\ p=", round(results[1], 3), "$")

\item [{[ 0 , 0 ]}]:  Adaptive $(M= 0.086 ,\ SD= 0.007 )$ , Zero $(M= 0.086 ,\ SD= 0.006 ),\ t( 198 )= 0.36 ,\ p= 0.719 $
\item [{[ 0 , 1 ]}]:  Adaptive $(M= 0.543 ,\ SD= 0.049 )$ , Zero $(M= 0.535 ,\ SD= 0.04 ),\ t( 198 )= 1.38 ,\ p= 0.17 $
\item [{[ 0 , 2 ]}]:  Adaptive $(M= 0.089 ,\ SD= 0.007 )$ , Zero $(M= 0.086 ,\ SD= 0.006 ),\ t( 198 )= 3.29 ,\ p= 0.001 $
\item [{[ 1 , 0 ]}]:  Adaptive $(M= 0.531 ,\ SD= 0.05 )$ , Zero $(M= 0.533 ,\ SD= 0.042 ),\ t( 198 )= -0.28 ,\ p= 0.778 $
\item [{[ 1 , 1 ]}]:  Adaptive $(M= 2.632 ,\ SD= 0.135 )$ , Zero $(M= 3.564 ,\ SD= 0.086 ),\ t( 198 )= -58.13 ,\ p= 0.0 $
\item [{[ 1 , 2 ]}]:  Adaptive $(M= 0.637 ,\ SD= 0.071 )$ , Zero $(M= 0.635 ,\ SD= 0.045 ),\ t( 198 )= 0.24 ,\ p= 0.813 $
\item [{[ 2 , 0 ]}]:  Adaptive $(M= 0.089 ,\ SD= 0.005 )$ , Zero $(M= 0.088 ,\ SD= 0.007 ),\ t( 198 )= 1.89 ,\ p= 0.061 $
\item [{[ 2 , 1 ]}]:  Adaptive $(M= 0.584 ,\ SD= 0.048 )$ , Zero $(M= 0.592 ,\ SD= 0.043 ),\ t( 198 )= -1.22 ,\ p= 0.224 $
\item [{[ 2 , 2 ]}]:  Ada

In [113]:
# ###### Grid based###### DEPRECATED/NOT USED

# # Average time
# print("AVERAGE TIME WAITED")
# average_time_grid = []
# for exp in results_to_use:
#     average_time_grid.append(
#         all_sim_results[exp]['stat_average_time_waited_grid'])

# means_1 = []
# for i in range(len(average_time_grid[0])):
#     means_1.append(np.mean(average_time_grid[0][i]))
# means_2 = []
# for i in range(len(average_time_grid[1])):
#     means_2.append(np.mean(average_time_grid[1][i]))

# print(np.mean(average_time_grid[0], axis=0))
# # print("mean = ", np.mean(np.mean(average_time_grid[0], axis=0)))
# # print("std = ", np.std(np.mean(average_time_grid[0], axis=0)))

# results = stats.ttest_ind(means_1, means_2)
# print(results)
# print("The average time waited for the first two simulations are: " +
#       str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
# print("The p-value is: " + str(results[1]))
# mean_1 = round(np.mean(means_1), 3)
# mean_2 = round(np.mean(means_2), 3)
# std_1 = round(np.std(means_1), 3)
# std_2 = round(np.std(means_2), 3)
# print("in latex:", "$(M=", mean_1, ",\ SD=", std_1, ")$",
#       "$(M=", mean_2, ",\ SD=", std_2, ")$")
# print("in latex:", "$t(", results.df, ")=", round(
#     results[0], 3), ", p=", round(results[1], 3), "$")

# # Max time
# print("MAX TIME WAITED")
# max_time_grid = []
# for exp in results_to_use:
#     max_time_grid.append(
#         all_sim_results[exp]['stat_max_time_waited_grid'])

# means_1 = []
# for i in range(len(max_time_grid[0])):
#     means_1.append(np.mean(max_time_grid[0][i]))
# means_2 = []
# for i in range(len(max_time_grid[1])):
#     means_2.append(np.mean(max_time_grid[1][i]))


# results = stats.ttest_ind(means_1, means_2)
# print(results)
# print("The max time waited for the first two simulations are: " +
#       str(np.mean(means_1)) + " and " + str(np.mean(means_2)))
# print("The p-value is: " + str(results[1]))
# mean_1 = round(np.mean(means_1), 3)
# mean_2 = round(np.mean(means_2), 3)
# std_1 = round(np.std(means_1), 3)
# std_2 = round(np.std(means_2), 3)
# print("in latex:", "$(M=", mean_1, ",\ SD=", std_1, ")$",
#       "$(M=", mean_2, ",\ SD=", std_2, ")$")
# print("in latex:", "$t(", results.df, ")=", round(
#     results[0], 3), ", p=", round(results[1], 3), "$")

## 4. Gini coefficient <a id="4"></a>

In [114]:
# Average Time Waited.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_time_waited = []
for sim in results_to_use:
    all_ginis_time_waited.append(
        all_sim_results[sim]['stat_time_waited_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
# Present the results nicely
print("The average gini of time waited for the first two simulations are: " + str(np.mean(all_ginis_time_waited[0])) + " and " + str(np.mean(all_ginis_time_waited[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(all_ginis_time_waited[0]),3)
mean_2 = round(np.mean(all_ginis_time_waited[1]),3)
std_1 = round(np.std(all_ginis_time_waited[0]),3)
std_2 = round(np.std(all_ginis_time_waited[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")



TtestResult(statistic=11.724338842983359, pvalue=1.8841780908713807e-24, df=198.0)
The average gini of time waited for the first two simulations are: 0.13611684867377347 and 0.13208587337373903
The p-value is: 1.8841780908713807e-24
in latex: $(M= 0.136 ,\ SD= 0.003 )$ $(M= 0.132 ,\ SD= 0.002 )$
in latex: $t( 198.0 )= 11.724 , p= 0.0 $


In [115]:
# Average Satisfaction.
#Averaged over epochs and intersections: the gini of each intersection, and average of all the ginis per epoch.
all_ginis_satisfaction = []
for sim in results_to_use:
    all_ginis_satisfaction.append(
        all_sim_results[sim]['stat_satisfaction_gini'])

results = stats.ttest_ind(all_ginis_time_waited[0], all_ginis_time_waited[1])
print(results)
#present the results nicely
print("The average gini of satisfaction for the first two simulations are: " + str(np.mean(all_ginis_satisfaction[0])) + " and " + str(np.mean(all_ginis_satisfaction[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(all_ginis_satisfaction[0]),3)
mean_2 = round(np.mean(all_ginis_satisfaction[1]),3)
std_1 = round(np.std(all_ginis_satisfaction[0]),3)
std_2 = round(np.std(all_ginis_satisfaction[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")



TtestResult(statistic=11.724338842983359, pvalue=1.8841780908713807e-24, df=198.0)
The average gini of satisfaction for the first two simulations are: 0.416427199608096 and 0.42382260423539814
The p-value is: 1.8841780908713807e-24
in latex: $(M= 0.416 ,\ SD= 0.002 )$ $(M= 0.424 ,\ SD= 0.002 )$
in latex: $t( 198.0 )= 11.724 , p= 0.0 $


## 5. Trip Satisfaction <a id="5"></a>

In [116]:
all_satisfactions = []
for sim in results_to_use:
    all_satisfactions.append(
        all_sim_results[sim]['stat_satisfaction_mean'])

results = stats.ttest_ind(all_satisfactions[0], all_satisfactions[1])
print(results)
#present the results nicely
print("The average satisfaction for the first two simulations are: " + str(np.mean(all_satisfactions[0])) + " and " + str(np.mean(all_satisfactions[1])))
print("The p-value is: " + str(results[1]))
mean_1 = round(np.mean(all_satisfactions[0]),3)
mean_2 = round(np.mean(all_satisfactions[1]),3)
std_1 = round(np.std(all_satisfactions[0]),3)
std_2 = round(np.std(all_satisfactions[1]),3)
print("in latex:","$(M=",mean_1,",\ SD=",std_1,")$","$(M=",mean_2,",\ SD=",std_2,")$")
print("in latex:", "$t(", results.df, ")=", round(results[0], 3), ", p=", round(results[1], 3), "$")


TtestResult(statistic=7.855875627496529, pvalue=2.472216705568414e-13, df=198.0)
The average satisfaction for the first two simulations are: 0.16730737462014328 and 0.16635187668329274
The p-value is: 2.472216705568414e-13
in latex: $(M= 0.167 ,\ SD= 0.001 )$ $(M= 0.166 ,\ SD= 0.001 )$
in latex: $t( 198.0 )= 7.856 , p= 0.0 $


## 6. Auction Reward <a id="5"></a>

In [117]:
######Intersection based - Average Auction Reward######

average_reward_intersection = []
for exp in results_to_use:
    average_reward_intersection.append(
        all_sim_results[exp]['stat_average_auction_reward_per_intersection'])
    
for i in range(len(average_reward_intersection[0][0])):
    for j in range(len(average_reward_intersection[0][0])):
        print("Intersection [", i, "][", j, "]")
        results = stats.ttest_ind(average_reward_intersection[0][:,i,j], average_reward_intersection[1][:,i,j])
        print(results)
        print("The average auction reward for the first two simulations are: " + str(np.mean(average_reward_intersection[0][:,i,j])) + " and " + str(np.mean(average_reward_intersection[1][:,i,j])))
        print("The p-value is: " + str(results[1]))

Intersection [ 0 ][ 0 ]
TtestResult(statistic=1.26675229369833, pvalue=0.20673195066741323, df=198.0)
The average auction reward for the first two simulations are: 0.05584549999999999 and 0.055218833333333335
The p-value is: 0.20673195066741323
Intersection [ 0 ][ 1 ]
TtestResult(statistic=4.555973912544793, pvalue=9.107443621394974e-06, df=198.0)
The average auction reward for the first two simulations are: 0.24500391666666663 and 0.23798366666666668
The p-value is: 9.107443621394974e-06
Intersection [ 0 ][ 2 ]
TtestResult(statistic=3.744308202810813, pvalue=0.0002370712228415897, df=198.0)
The average auction reward for the first two simulations are: 0.057355166666666665 and 0.0555785
The p-value is: 0.0002370712228415897
Intersection [ 1 ][ 0 ]
TtestResult(statistic=4.362035436248009, pvalue=2.071620072202795e-05, df=198.0)
The average auction reward for the first two simulations are: 0.24614458333333328 and 0.23951475
The p-value is: 2.071620072202795e-05
Intersection [ 1 ][ 1 ]
Tt